In [129]:
## Code courtsey: Manav Sehgal (https://www.kaggle.com/startupsci)
## Link to Original Code: https://www.kaggle.com/startupsci/titanic-data-science-solutions

# data analysis and wrangling

#Data cleaning script taken from :https://gist.github.com/jay-trivedi/4ab63ebf09313094e8e97376d6f254ad

import pandas as pd
import numpy as np
import random as rnd

def titanic(train, test):
    train_df = pd.read_csv(train)
    test_df = pd.read_csv(test)
    combine = [train_df, test_df]

    train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
    test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
    combine = [train_df, test_df]

    for dataset in combine:
        dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

    pd.crosstab(train_df['Title'], train_df['Sex'])

    for dataset in combine:
        dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
        'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

        dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
        dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
        dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

    train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    for dataset in combine:
        dataset['Title'] = dataset['Title'].map(title_mapping)
        dataset['Title'] = dataset['Title'].fillna(0)

    train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
    test_df = test_df.drop(['Name'], axis=1)
    combine = [train_df, test_df]

    for dataset in combine:
        dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

    guess_ages = np.zeros((2,3))
    guess_ages

    for dataset in combine:
        for i in range(0, 2):
            for j in range(0, 3):
                guess_df = dataset[(dataset['Sex'] == i) & \
                                      (dataset['Pclass'] == j+1)]['Age'].dropna()

                # age_mean = guess_df.mean()
                # age_std = guess_df.std()
                # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

                age_guess = guess_df.median()

                # Convert random age float to nearest .5 age
                guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5

        for i in range(0, 2):
            for j in range(0, 3):
                dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                        'Age'] = guess_ages[i,j]

        dataset['Age'] = dataset['Age'].astype(int)

    for dataset in combine:
        dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1



    for dataset in combine:
        dataset['IsAlone'] = 0
        dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

    combine = [train_df, test_df]

    freq_port = train_df.Embarked.dropna().mode()[0]

    for dataset in combine:
        dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)

    test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)

    test_df.head(10)
    return train_df,test_df

In [130]:
train_df,test_df = titanic('./train.csv','./titanic_test.csv')

In [131]:
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize,IsAlone
0,0,3,0,22,1,0,7.2500,S,1,2,0
1,1,1,1,38,1,0,71.2833,C,3,2,0
2,1,3,1,26,0,0,7.9250,S,2,1,1
3,1,1,1,35,1,0,53.1000,S,3,2,0
4,0,3,0,35,0,0,8.0500,S,1,1,1
5,0,3,0,25,0,0,8.4583,Q,1,1,1
6,0,1,0,54,0,0,51.8625,S,1,1,1
7,0,3,0,2,3,1,21.0750,S,4,5,0
8,1,3,1,27,0,2,11.1333,S,3,3,0
9,1,2,1,14,1,0,30.0708,C,3,2,0


In [132]:
test_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize,IsAlone
0,892,3,0,34,0,0,7.8292,Q,1,1,1
1,893,3,1,47,1,0,7.0000,S,3,2,0
2,894,2,0,62,0,0,9.6875,Q,1,1,1
3,895,3,0,27,0,0,8.6625,S,1,1,1
4,896,3,1,22,1,1,12.2875,S,3,3,0
5,897,3,0,14,0,0,9.2250,S,1,1,1
6,898,3,1,30,0,0,7.6292,Q,2,1,1
7,899,2,0,26,1,1,29.0000,S,1,3,0
8,900,3,1,18,0,0,7.2292,C,3,1,1
9,901,3,0,21,2,0,24.1500,S,1,3,0


In [133]:
print len(train_df),len(test_df)
train_df[train_df.columns.tolist()[1:]]

891 418


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize,IsAlone
0,3,0,22,1,0,7.2500,S,1,2,0
1,1,1,38,1,0,71.2833,C,3,2,0
2,3,1,26,0,0,7.9250,S,2,1,1
3,1,1,35,1,0,53.1000,S,3,2,0
4,3,0,35,0,0,8.0500,S,1,1,1
5,3,0,25,0,0,8.4583,Q,1,1,1
6,1,0,54,0,0,51.8625,S,1,1,1
7,3,0,2,3,1,21.0750,S,4,5,0
8,3,1,27,0,2,11.1333,S,3,3,0
9,2,1,14,1,0,30.0708,C,3,2,0


In [134]:
train_df = pd.get_dummies(train_df)
train_df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Title,FamilySize,IsAlone,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22,1,0,7.2500,1,2,0,0,0,1
1,1,1,1,38,1,0,71.2833,3,2,0,1,0,0
2,1,3,1,26,0,0,7.9250,2,1,1,0,0,1
3,1,1,1,35,1,0,53.1000,3,2,0,0,0,1
4,0,3,0,35,0,0,8.0500,1,1,1,0,0,1
5,0,3,0,25,0,0,8.4583,1,1,1,0,1,0
6,0,1,0,54,0,0,51.8625,1,1,1,0,0,1
7,0,3,0,2,3,1,21.0750,4,5,0,0,0,1
8,1,3,1,27,0,2,11.1333,3,3,0,0,0,1
9,1,2,1,14,1,0,30.0708,3,2,0,1,0,0


In [135]:
test_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize,IsAlone
0,892,3,0,34,0,0,7.8292,Q,1,1,1
1,893,3,1,47,1,0,7.0000,S,3,2,0
2,894,2,0,62,0,0,9.6875,Q,1,1,1
3,895,3,0,27,0,0,8.6625,S,1,1,1
4,896,3,1,22,1,1,12.2875,S,3,3,0
5,897,3,0,14,0,0,9.2250,S,1,1,1
6,898,3,1,30,0,0,7.6292,Q,2,1,1
7,899,2,0,26,1,1,29.0000,S,1,3,0
8,900,3,1,18,0,0,7.2292,C,3,1,1
9,901,3,0,21,2,0,24.1500,S,1,3,0


In [136]:
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report
from sklearn import tree

tclf = tree.DecisionTreeClassifier(max_depth=2)
tclf.fit(train_df[train_df.columns.tolist()[1:]],train_df['Survived'])


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [137]:
test_df = pd.get_dummies(test_df)
print test_df.columns

Index([u'PassengerId', u'Pclass', u'Sex', u'Age', u'SibSp', u'Parch', u'Fare',
       u'Title', u'FamilySize', u'IsAlone', u'Embarked_C', u'Embarked_Q',
       u'Embarked_S'],
      dtype='object')


In [138]:
print len(test_df.columns)
p_var = test_df['PassengerId']
del test_df['PassengerId']
print len(train_df.columns)
print len(test_df.columns)

13
13
12


In [139]:
s_var = tclf.predict(test_df)

In [140]:
f = open('./submit.csv','w+')
zipped = zip(p_var,s_var)
f.write('PassengerId,Survived\n')
for e in zipped:
    f.write('{},{}\n'.format(e[0],e[1]))
f.close()

In [142]:
df_op = pd.DataFrame(np.array([p_var,s_var]).T,columns=['PassengerId','Survived'])
df_op.to_csv('df_csv.csv',index=False)